# Analyzing Travel Times and Distances

This gathers travel times and distances using the [Bing Maps Distance Matrix API](https://www.microsoft.com/en-us/maps/distance-matrix).

Given a list of $O$ origins and $D$ destinations, it calculates a $O$ by $D$ matrix of the respective travel time and distance for each origin and distance. The service [limits our queries](https://learn.microsoft.com/en-us/bingmaps/rest-services/routes/calculate-a-distance-matrix#api-limits) to "$2,500$ origins-destinations pairs," i.e $O \times D \leq 2500$. This should be more than enough for our use case. (We're also arbitrarily rate limited, hence the `sleep` timer.)

Ensure that Python 3.x and the following dependencies are installed. There is a `requirements.txt` file in the root reporitostory directory that you can install via `pip install -r requirements.txt`, though it might be outdated.

In [66]:
from config import MAPS_API_KEY as API_KEY
import requests
from requests import Response
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np
from time import sleep
from pprint import pprint
import asyncio
from aiohttp import ClientSession

import plotly.graph_objects as go
import plotly.express as px

### Example Query

Below is an example API query: multiple origins, one destination.
Here, both origins and destinations are in latitude and longitude coordinates (in that order). The URL essentially accepts these as semi-colon-separated pairs.

**Note:** In our actual querying, we will be relying on `aiohttp`'s `request` to support multi-threading instead of the `requests` shown here.

In [3]:
orig_lat_0, orig_long_0 = 36.03, -111.44
orig_lat_1, orig_long_1 = 36.28, -111.68
dest_lat, dest_long = 36.63, -111.65

ex_query_url = f"https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?origins={orig_lat_0},{orig_long_0};{orig_lat_1},{orig_long_1}&destinations={dest_lat},{dest_long}&travelMode=driving&key={API_KEY}"
ex_query = requests.request("GET", ex_query_url)
pprint(ex_query.json())

{'authenticationResultCode': 'ValidCredentials',
 'brandLogoUri': 'http://dev.virtualearth.net/Branding/logo_powered_by.png',
 'copyright': 'Copyright © 2023 Microsoft and its suppliers. All rights '
              'reserved. This API cannot be accessed and the content and any '
              'results may not be used, reproduced or transmitted in any '
              'manner without express written permission from Microsoft '
              'Corporation.',
 'resourceSets': [{'estimatedTotal': 1,
                   'resources': [{'__type': 'DistanceMatrix:http://schemas.microsoft.com/search/local/ws/rest/v1',
                                  'destinations': [{'latitude': 36.63,
                                                    'longitude': -111.65}],
                                  'origins': [{'latitude': 36.03,
                                               'longitude': -111.44},
                                              {'latitude': 36.28,
                                      

### Formatting Locations Into API-Specified String
Accepts a DataFrame input (see "Reading and Filtering Input Data") and formats its latitude and loingitude coordinates to prepare it for querying.

In [4]:
def format_locations_list(locs: pd.DataFrame) -> list[str]:
    return [
        f"{coords['LATITUDE']},{coords['LONGITUDE']}" for _, coords in locs.iterrows()
    ]


def format_locations_str(locs: pd.DataFrame) -> str:
    return ";".join(format_locations_list(locs))


def format_location_str(lat: float, long: float) -> str:
    return f"{lat},{long};"

## Homes and Corrals vs. Watering Points

### Reading and Filtering Input Data
Most (if not all) of customizing the query is here, as we get to specify which destinations and origins to calculate distances against.

In [87]:
df = pd.read_excel("Water Optimization/travel_analysis/Master_sheet_oct_22.xlsx")
locs = df[["NAME", "TYPE", "LONGITUDE", "LATITUDE"]]

# Filter to homes and corrals (320 total, as of 4/13/2023)
# Requires bitwise OR `|` and paranthesis around each conditional, i.e. `(locs['Column name'] == 'Row value") | ...`
homes_corrals = locs.loc[(locs["NAME"] == "Home") | (locs["NAME"] == "Corral")][:150]

# Filter to reservoir candidates (137 total, as of 4/13/2023)
reservoirs = locs[
    (locs["TYPE"] == "LW Groundwater Source")
    | (locs["TYPE"] == "PW Groundwater Source")
    | (locs["TYPE"] == "LW Surface Water Source")
][:50]

print(
    f"Extracted {len(homes_corrals)} origins (homes/corrals) and {len(reservoirs)} destinations (reservoirs)."
)

Extracted 150 origins (homes/corrals) and 50 destinations (reservoirs).


### Parallel Querying

In [88]:
# def fetch_distances(formatted_dests: str, formatted_origs: str) -> Response:
#     """
#     Given the formatted origins and destinations, returns a { request: str, url: str } from the Bing Maps API
#     """
#     link = f"https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?origins={formatted_origs}&destinations={formatted_dests}&travelMode=driving&key={API_KEY}"
#     return requests.request("GET", link)


async def _get_response(session: ClientSession, url: str) -> dict:
    async with session.get(url) as response:
        return await response.json()


async def _make_request(
    url: str, queue: asyncio.Queue, semaphore: asyncio.Semaphore
) -> None:
    async with semaphore:
        async with ClientSession() as session:
            too_many_reqs = True
            response = dict()

            while too_many_reqs:
                response = await _get_response(session, url)
                if response["statusCode"] != 429:
                    too_many_reqs = False
                else:
                    await asyncio.sleep(0.001)

            await queue.put({"response": response, "url": url})


async def run_query(origs: pd.DataFrame, dests: pd.DataFrame) -> list:
    results = []

    queue = asyncio.Queue()
    semaphore = asyncio.Semaphore(50)

    formatted_origs = format_locations_list(origs)
    formatted_dests = format_locations_list(dests)

    async with asyncio.TaskGroup() as group:
        for orig in formatted_origs:
            for dest in formatted_dests:
                group.create_task(
                    _make_request(
                        f"https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?origins={orig}&destinations={dest}&travelMode=driving&key={API_KEY}",
                        queue,
                        semaphore,
                    )
                )

    count = 0
    while not queue.empty():
        if count >= 50:
            await asyncio.sleep(0.001)
            count = 0

        results.append(await queue.get())
        count += 1

    return results

### Querying Command (Run conservatively)

In [89]:
# Entire query is in one (the first) result
results = (await asyncio.gather(run_query(homes_corrals, reservoirs)))[0]
pprint(results)

ExceptionGroup: unhandled errors in a TaskGroup (1 sub-exception)

## Processing the Results
### Example Formatted Result

In [28]:
[
    {
        "authenticationResultCode": "ValidCredentials",
        "brandLogoUri": "http://dev.virtualearth.net/Branding/logo_powered_by.png",
        "copyright": "Copyright © 2023 Microsoft and its suppliers. All rights "
        "reserved. This API cannot be accessed and the content and any "
        "results may not be used, reproduced or transmitted in any "
        "manner without express written permission from Microsoft "
        "Corporation.",
        "resourceSets": [
            {
                "estimatedTotal": 1,
                "resources": [
                    {
                        "__type": "DistanceMatrix:http://schemas.microsoft.com/search/local/ws/rest/v1",
                        "destinations": [{"latitude": 36.63, "longitude": -111.65}],
                        "origins": [
                            {"latitude": 36.03, "longitude": -111.44},
                            {"latitude": 36.28, "longitude": -111.68},
                        ],
                        "results": [
                            {
                                "destinationIndex": 0,
                                "originIndex": 0,
                                "totalWalkDuration": 0,
                                "travelDistance": 73.363,
                                "travelDuration": 40.4833,
                            },
                            {
                                "destinationIndex": 0,
                                "originIndex": 1,
                                "totalWalkDuration": 0,
                                "travelDistance": 53.429,
                                "travelDuration": 65.7,
                            },
                        ],
                    }
                ],
            }
        ],
        "statusCode": 200,
        "statusDescription": "OK",
        "traceId": "d2b590f1fcb84de2be0af4d7ddae9c5c|CH0001B043|0.0.0.0|CHI001B78C",
    }
    # ...
]

https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?origins=36.0057708,-111.5161282;36.051963,-111.5421178;36.4005527,-111.7203825;36.3403401,-111.7622694;36.0638527,-111.5355458;36.5608292,-111.7391979;36.5232688,-111.728044;36.0645147,-111.6339967;36.1145249,-111.6159235;36.250792,-111.7629831;36.3083024,-111.7512179;36.2758368,-111.6756069;36.2772374,-111.6828238;36.4569573,-111.691696;36.4090399,-111.6266555;36.5198745,-111.6644153;36.4246905,-111.5703322;36.2855293,-111.5073213;36.3052537,-111.5107852;36.6213637,-111.6767236;36.4322183,-111.6900709;36.2253496,-111.3958839;35.8951607,-111.4045696;35.889656,-111.4035683;35.8981984,-111.4000044;35.8934746,-111.4045267;36.2096925,-111.3968916;36.2180564,-111.4001999;36.2223209,-111.401304;36.2099348,-111.3983293;36.2113582,-111.398456;36.2601551,-111.4199452;36.3059606,-111.4670218;36.2490893,-111.431771;36.2588833,-111.4198674;36.2594176,-111.4199666;36.2734216,-111.4395327;36.2660638,-111.4249204;36.3078347,-111.4534645;36.30

### Exporting Raw Results

In [44]:
with open("Water Optimization/travel_analysis/bing_raw_results.json", "w") as outfile:
    outfile.write(json.dumps(results, indent=4))

In [77]:
for result in results:
    resource = result["resourceSets"][0]["resources"][0]
    dest = (
        resource["destinations"][0]["latitude"],
        resource["destinations"][0]["longitude"],
    )
    orig = (resource["origins"][0]["latitude"], resource["origins"][0]["longitude"])
    duration = resource["results"][0]["travelDuration"]
    pprint(resource)

### Formatting Results

In [ ]:
# To format into # of mins. e.g. input times "23 mins" or "1 hour 3 mins"
def format_time(time: str) -> int:
    min_index = time.index("min")
    if "hour" in time:
        hr_index = time.index("hour")
        time = int(time[:hr_index]) * 60 + int(time[hr_index + len("hour") : min_index])
    else:
        time = int(time[:min_index])

    return time

In [ ]:
# 341 (origins) X 5 (destinations)
time_matrix = np.zeros((len(origs), len(dests)), int)

orig_i = 0
for res_obj in raw_results:
    res = json.loads(res_obj.text)
    for i, dest_infos in enumerate(res["rows"]):
        for dest_i, travel_info in enumerate(res["rows"][i]["elements"]):
            time_matrix[orig_i, dest_i] = format_time(travel_info["duration"]["text"])
        orig_i += 1

for i, col in enumerate(time_matrix.T):
    origs.assign(**{dests["Name"].iloc[i] + " Travel Time (min.)": col})

origs.to_excel("formatted_data.xlsx", index=False)

### Computing Analyses

In [ ]:
median_times = np.median(time_matrix, axis=1)
mean_times = np.mean(time_matrix, axis=1)

origs = origs.assign(**{"Median": median_times.tolist()})
origs = origs.assign(**{"Mean": mean_times.tolist()})

origs.to_excel("formatted_data.xlsx", index=False)

### Analyses Figures
IDK how to neatly plot in the same figure, so two cells each for mean and median.

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot()
n, bins, patches = ax1.hist(
    median_times, bins=20, weights=np.ones(len(mean_times)) / len(mean_times)
)

plt.title("Homes and Corrals vs. Median Travel Time")
plt.xlabel("Median Travel Time to Watering Points (mins.)")
plt.ylabel("% of All Homes/Corrals")

In [ ]:
fig = plt.figure()
ax2 = fig.add_subplot()
n, bins, patches = ax2.hist(
    mean_times, bins=20, weights=np.ones(len(mean_times)) / len(mean_times)
)

plt.title("Homes and Corrals vs. Mean Travel Time")
plt.xlabel("Mean Travel Time to Watering Points (mins.)")
plt.ylabel("% of All Homes/Corrals")

In [ ]:
print(
    f"""
      | Other Information |
      Number of origins: {len(origs)}
      Number of destinations: {len(dests)}
      Standard deviation of median times: {np.std(median_times)}
      Standard deviation of mean times: {np.std(mean_times)}
      """
)

## Geoplotting

In [ ]:
boundary_points = None
with open("travel_analysis/boundary.txt") as f:
    point_strs = f.readlines()[0].split(", ")
    boundary_points = np.empty((len(point_strs), 2), dtype=tuple)
    for i, point_str in enumerate(point_strs):
        p = point_str.split(" ")
        boundary_points[i] = np.array([float(p[0]), float(p[1])])

In [ ]:
# Hover labels for plot points
orig_labels = (
    origs["Name"]
    + ", "
    + origs["Type"]
    + ". Min: "
    + time_matrix.min(axis=1).astype(str)
)
dest_labels = dests["Name"] + ", " + dests["Type"]

# Origins (Homes and Corrals)
fig = go.Figure(
    data=go.Scattergeo(
        # locationmode='USA-states',
        lat=origs["Latitude"],
        lon=origs["Longitude"],
        text=orig_labels,
        marker=dict(
            line=dict(width=1, color="rgba(102, 102, 102)"),
            # colorscale='Blues',
            cmin=0,
            color=time_matrix.min(axis=1),
            cmax=time_matrix.min(axis=1).max(),
            colorbar_title="Minimum Time (min.)",
        ),
    )
)

# Destinations (Watering Points)
fig.add_trace(
    go.Scattergeo(
        lat=dests["Latitude"],
        lon=dests["Longitude"],
        text=dest_labels,
        marker=dict(symbol="square", line=dict(width=1, color="rgba(102, 102, 102)")),
    )
)

# Chapter Border
fig.add_trace(
    go.Scattergeo(lat=boundary_points[:, 1], lon=boundary_points[:, 0], mode="lines")
)
fig.update_geos(fitbounds="locations")
fig.update_layout(legend_orientation="h", mapbox_style="open-street-map")
fig.show()

In [ ]:
fig = px.scatter_mapbox(
    origs,
    lat="Latitude",
    lon="Longitude",
    hover_name="Name",
    hover_data=["Type"],
    color_discrete_sequence=["fuchsia"],
)
fig.add_trace(
    go.Scattergeo(lat=boundary_points[:, 1], lon=boundary_points[:, 0], mode="lines")
)
fig.update_layout(
    margin={"r": 0, "t": 0, "l": 0, "b": 0}, mapbox_style="open-street-map"
)
fig.show()

In [ ]:
print(orig_labels)

In [ ]:
fig = go.Figure(
    data=go.Contour(
        z=time_matrix.min(axis=1),
        x=origs["Latitude"],
        y=origs["Longitude"],
        colorscale="Hot",
        contours_coloring="heatmap",
    )
)

fig.show()

## Countour Plot

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Carpet(
        a=[0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3],
        b=[4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6],
        x=[2, 3, 4, 5, 2.2, 3.1, 4.1, 5.1, 1.5, 2.5, 3.5, 4.5],
        y=[1, 1.4, 1.6, 1.75, 2, 2.5, 2.7, 2.75, 3, 3.5, 3.7, 3.75],
        aaxis=dict(tickprefix="a = ", smoothing=0, minorgridcount=9, type="linear"),
        baxis=dict(tickprefix="b = ", smoothing=0, minorgridcount=9, type="linear"),
    )
)
fig.update_layout()
fig.show()